In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [48]:
path_to_file = "/poems/PG4741_text.txt0.txt"

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 121 characters


In [49]:
# Take a look at the first 250 characters in text
print(text[:250])

Au mouvement des sources
nos mères spongieuses
à la terre
enracinées
s'agrippent aux foetus
déjà mûrs
de l'automne achevé


In [50]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

28 unique characters


In [60]:
# Vectoriser le texte
# Avant la formation, vous devez convertir les chaînes en une représentation numérique.
# La couche tf.keras.layers.StringLookup peut convertir chaque caractère en un ID numérique.
# Il a juste besoin que le texte soit d'abord divisé en jetons.

example_texts = text.split()

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'A', b'u'], [b'm', b'o', b'u', b'v', b'e', b'm', b'e', b'n', b't'],
 [b'd', b'e', b's'], [b's', b'o', b'u', b'r', b'c', b'e', b's'],
 [b'n', b'o', b's'], [b'm', b'\xc3\xa8', b'r', b'e', b's'],
 [b's', b'p', b'o', b'n', b'g', b'i', b'e', b'u', b's', b'e', b's'],
 [b'\xc3\xa0'], [b'l', b'a'], [b't', b'e', b'r', b'r', b'e'],
 [b'e', b'n', b'r', b'a', b'c', b'i', b'n', b'\xc3\xa9', b'e', b's'],
 [b's', b"'", b'a', b'g', b'r', b'i', b'p', b'p', b'e', b'n', b't'],
 [b'a', b'u', b'x'], [b'f', b'o', b'e', b't', b'u', b's'],
 [b'd', b'\xc3\xa9', b'j', b'\xc3\xa0'], [b'm', b'\xc3\xbb', b'r', b's'],
 [b'd', b'e'], [b'l', b"'", b'a', b'u', b't', b'o', b'm', b'n', b'e'],
 [b'a', b'c', b'h', b'e', b'v', b'\xc3\xa9']]>

In [6]:
# Vectoriser le texte
# Avant la formation, vous devez convertir les chaînes en une représentation numérique.
# La couche tf.keras.layers.StringLookup peut convertir chaque caractère en un ID numérique.
# Il a juste besoin que le texte soit d'abord divisé en jetons.

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [7]:
# Il convertit les jetons en ID de personnage
ids = ids_from_chars(chars)

In [8]:
# Le but de ce tutoriel étant de générer du texte,
# il sera également important d'inverser cette représentation
# et d'en récupérer les chaînes lisibles par l'homme.
# Pour cela, vous pouvez utiliser tf.keras.layers.StringLookup(..., invert=True) .

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [9]:
# Cette couche récupère les caractères des vecteurs d'IDs,
# et les renvoie sous la forme d'un tf.RaggedTensor de caractères :

chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'A', b'u'], [b'm', b'o', b'u', b'v', b'e', b'm', b'e', b'n', b't'],
 [b'd', b'e', b's'], [b's', b'o', b'u', b'r', b'c', b'e', b's'],
 [b'n', b'o', b's'], [b'm', b'\xc3\xa8', b'r', b'e', b's'],
 [b's', b'p', b'o', b'n', b'g', b'i', b'e', b'u', b's', b'e', b's'],
 [b'\xc3\xa0'], [b'l', b'a'], [b't', b'e', b'r', b'r', b'e'],
 [b'e', b'n', b'r', b'a', b'c', b'i', b'n', b'\xc3\xa9', b'e', b's'],
 [b's', b"'", b'a', b'g', b'r', b'i', b'p', b'p', b'e', b'n', b't'],
 [b'a', b'u', b'x'], [b'f', b'o', b'e', b't', b'u', b's'],
 [b'd', b'\xc3\xa9', b'j', b'\xc3\xa0'], [b'm', b'\xc3\xbb', b'r', b's'],
 [b'd', b'e'], [b'l', b"'", b'a', b'u', b't', b'o', b'm', b'n', b'e'],
 [b'a', b'c', b'h', b'e', b'v', b'\xc3\xa9']]>

In [10]:
# Vous pouvez tf.strings.reduce_join pour joindre les caractères dans des chaînes.

tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'Au', b'mouvement', b'des', b'sources', b'nos', b'm\xc3\xa8res',
       b'spongieuses', b'\xc3\xa0', b'la', b'terre', b'enracin\xc3\xa9es',
       b"s'agrippent", b'aux', b'foetus', b'd\xc3\xa9j\xc3\xa0',
       b'm\xc3\xbbrs', b'de', b"l'automne", b'achev\xc3\xa9'],
      dtype=object)

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


In [12]:
# Prédiction

# Divisez ensuite le texte en exemples de séquences. 
# Chaque séquence d'entrée contiendra seq_length caractères du texte.

# Pour chaque séquence d'entrée,
# les cibles correspondantes contiennent la même longueur de texte,
# sauf qu'elles sont décalées d'un caractère vers la droite.

# Divisez donc le texte en morceaux de seq_length+1 .
# Par exemple, disons que seq_length est 4 et que notre texte est "Bonjour".
# La séquence d'entrée serait "Hell" et la séquence cible "ello".

# Pour ce faire, utilisez d'abord la fonction tf.data.Dataset.from_tensor_slices
# pour convertir le vecteur de texte en un flux d'indices de caractères.

print(text)

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

Au mouvement des sources
nos mères spongieuses
à la terre
enracinées
s'agrippent aux foetus
déjà mûrs
de l'automne achevé


<tf.Tensor: shape=(121,), dtype=int64, numpy=
array([ 4, 22,  2, 15, 17, 22, 23,  8, 15,  8, 16, 21,  2,  7,  8, 20,  2,
       20, 17, 22, 19,  6,  8, 20,  1, 16, 17, 20,  2, 15, 26, 19,  8, 20,
        2, 20, 18, 17, 16, 10, 12,  8, 22, 20,  8, 20,  1, 25,  2, 14,  5,
        2, 21,  8, 19, 19,  8,  1,  8, 16, 19,  5,  6, 12, 16, 27,  8, 20,
        1, 20,  3,  5, 10, 19, 12, 18, 18,  8, 16, 21,  2,  5, 22, 24,  2,
        9, 17,  8, 21, 22, 20,  1,  7, 27, 13, 25,  2, 15, 28, 19, 20,  1,
        7,  8,  2, 14,  3,  5, 22, 21, 17, 15, 16,  8,  2,  5,  6, 11,  8,
       23, 27])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))


A
u
 
m
o
u
v
e
m
e


In [15]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
examples_per_epoch

1

In [16]:
# La méthode batch vous permet de convertir facilement ces caractères individuels
# en séquences de la taille souhaitée.

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))


tf.Tensor(
[b'A' b'u' b' ' b'm' b'o' b'u' b'v' b'e' b'm' b'e' b'n' b't' b' ' b'd'
 b'e' b's' b' ' b's' b'o' b'u' b'r' b'c' b'e' b's' b'\n' b'n' b'o' b's'
 b' ' b'm' b'\xc3\xa8' b'r' b'e' b's' b' ' b's' b'p' b'o' b'n' b'g' b'i'
 b'e' b'u' b's' b'e' b's' b'\n' b'\xc3\xa0' b' ' b'l' b'a' b' ' b't' b'e'
 b'r' b'r' b'e' b'\n' b'e' b'n' b'r' b'a' b'c' b'i' b'n' b'\xc3\xa9' b'e'
 b's' b'\n' b's' b"'" b'a' b'g' b'r' b'i' b'p' b'p' b'e' b'n' b't' b' '
 b'a' b'u' b'x' b' ' b'f' b'o' b'e' b't' b'u' b's' b'\n' b'd' b'\xc3\xa9'
 b'j' b'\xc3\xa0' b' ' b'm' b'\xc3\xbb' b'r' b's'], shape=(101,), dtype=string)


In [17]:
# Il est plus facile de voir ce que cela fait
# si vous rejoignez les jetons dans des chaînes

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"Au mouvement des sources\nnos m\xc3\xa8res spongieuses\n\xc3\xa0 la terre\nenracin\xc3\xa9es\ns'agrippent aux foetus\nd\xc3\xa9j\xc3\xa0 m\xc3\xbbrs"


In [18]:
# Pour la formation, vous aurez besoin d'un ensemble de données de paires (input, label) .
# Où input et l' label sont des séquences.
# A chaque pas de temps, l'entrée est le caractère courant et l'étiquette est le caractère suivant.

# Voici une fonction qui prend une séquence en entrée,
# la duplique et la décale pour aligner l'entrée et l'étiquette pour chaque pas de temps :

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [19]:
dataset = sequences.map(split_input_target)

In [20]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


Input : b"Au mouvement des sources\nnos m\xc3\xa8res spongieuses\n\xc3\xa0 la terre\nenracin\xc3\xa9es\ns'agrippent aux foetus\nd\xc3\xa9j\xc3\xa0 m\xc3\xbbr"
Target: b"u mouvement des sources\nnos m\xc3\xa8res spongieuses\n\xc3\xa0 la terre\nenracin\xc3\xa9es\ns'agrippent aux foetus\nd\xc3\xa9j\xc3\xa0 m\xc3\xbbrs"


In [22]:
# Créer des lots de formation
# Vous avez utilisé tf.data pour diviser le texte en séquences gérables.
# Mais avant d'introduire ces données dans le modèle,
# vous devez mélanger les données et les regrouper en lots.

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 256

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 100), dtype=tf.int64, name=None), TensorSpec(shape=(None, 100), dtype=tf.int64, name=None))>

In [23]:
# Construire le modèle

# Cette section définit le modèle comme une sous-classe keras.Model
# (pour plus de détails, voir Création de nouveaux calques et modèles via la sous-classe ).

# Ce modèle comporte trois couches :

# tf.keras.layers.Embedding : La couche d'entrée. Une table de recherche entraînable qui mappera chaque ID de caractère à un vecteur avec des dimensions embedding_dim ;
# tf.keras.layers.GRU : Un type de RNN avec des units=rnn_units (Vous pouvez également utiliser une couche LSTM ici.)
# tf.keras.layers.Dense : la couche de sortie, avec les sorties vocab_size . Il produit un logit pour chaque caractère du vocabulaire. Ce sont les log-vraisemblance de chaque caractère selon le modèle.

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
# Essayez le modèle

# Exécutez maintenant le modèle pour voir qu'il se comporte comme prévu.

# Vérifiez d'abord la forme de la sortie :

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(1, 100, 29) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  7424      
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  29725     
                                                                 
Total params: 3,975,453
Trainable params: 3,975,453
Non-trainable params: 0
_________________________________________________________________


In [27]:
# To get actual predictions from the model you need to sample from the output distribution,
# to get actual character indices.
# This distribution is defined by the logits over the character vocabulary.

# Try it for the first example in the batch:

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

# This gives us, at each timestep, a prediction of the next character index:

sampled_indices

array([24, 11, 28, 27, 10, 14,  8,  8,  3,  6, 18,  1, 20,  5,  2, 23,  4,
       17,  8, 20, 12, 23, 24, 12, 21, 28, 11,  0,  0, 25,  8, 26,  3, 28,
       19, 15,  5, 20, 14,  7, 19, 17, 24, 11,  5, 14,  3, 25, 20, 11,  6,
       19, 23,  0, 18,  1, 10, 18, 14,  8, 25, 18, 24, 14,  3,  5,  8, 11,
       15, 26, 13,  7,  5,  9, 17, 18, 24, 18,  4, 20, 13, 17, 11,  1,  4,
       27,  9, 17, 12,  2,  8, 21, 18,  1,  9,  4, 27, 16, 26, 17])

In [63]:
# Decode these to see the text predicted by this untrained model:

print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())
print()
print("Décodé:\n", text_from_ids(sampled_indices).numpy().decode("UTF-8"))

Input:
 b"Au mouvement des sources\nnos m\xc3\xa8res spongieuses\n\xc3\xa0 la terre\nenracin\xc3\xa9es\ns'agrippent aux foetus\nd\xc3\xa9j\xc3\xa0 m\xc3\xbbr"

Next Char Predictions:
 b"xh\xc3\xbb\xc3\xa9glee'cp\nsa vAoesivxit\xc3\xbbh[UNK][UNK]\xc3\xa0e\xc3\xa8'\xc3\xbbrmasldroxhal'\xc3\xa0shcrv[UNK]p\ngple\xc3\xa0pxl'aehm\xc3\xa8jdafopxpAsjoh\nA\xc3\xa9foi etp\nfA\xc3\xa9n\xc3\xa8o"

Décodé:
 xhûéglee'cp
sa vAoesivxitûh[UNK][UNK]àeè'ûrmasldroxhal'àshcrv[UNK]p
gpleàpxl'aehmèjdafopxpAsjoh
Aéfoi etp
fAénèo
